In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import random
import pathlib

In [ ]:
# Check for GPU availability
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    print('GPU is available')
else:
    print('No GPU detected')

num_gpus = len(physical_devices)

if num_gpus > 0:
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        print(f"GPU {i}: {tf.config.experimental.get_device_details(physical_devices[0])}")
else:
    print("No GPUs available")

device = tf.device('gpu:0' if len(physical_devices) > 0 else 'cpu:0')

In [ ]:
import os
os.getcwd()

Task: 01

In [ ]:
data_dir = 'd:\\SoftCom_Assignment01\\Dataset'
os.listdir(data_dir)

Task: 02 & 03

In [ ]:
tf.random.set_seed(42)

train_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'training', seed = 1, shuffle = True, batch_size = 32, image_size=(256,256))

test_data = keras.utils.image_dataset_from_directory(data_dir, validation_split = 0.1, subset = 'validation', seed = 1, shuffle = True, batch_size = 32, image_size=(256,256))

In [ ]:
filenames = pathlib.Path(data_dir)
for label in train_data.class_names :
    images = list(filenames.glob(f'{label}/*'))
    print(f'{label} : {len(images)}')

In [ ]:
train_data.cardinality().numpy(),  test_data.cardinality().numpy()

In [ ]:
train_set = train_data.take(130)
val_set = train_data.skip(130)

In [ ]:
train_set.cardinality().numpy(), val_set.cardinality().numpy()

In [ ]:
# print random images from the train set
plt.figure(figsize = (15, 15))
for images, labels in train_set.take(1):
    for i in range(15):
        index = random.randint(0, len(images))
        ax = plt.subplot(3, 5, i + 1)
        plt.imshow(images[index].numpy().astype("uint8"))
        plt.title(train_data.class_names[labels[index]], color= 'blue', fontsize= 12)
        plt.axis(True)
plt.show()

In [ ]:
for images_batch, labels_batch in train_set:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

Task: 04 & 05

In [ ]:
from tensorflow.keras import layers

In [ ]:
tf.random.set_seed(42)

model = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(128,(3,3), activation='relu', input_shape=(256,256,3)), # Hidden Layer 1
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(64,(3,3), activation='relu'), # Hidden Layer 2
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(32,(3,3), activation='relu'), # Hidden Layer 3
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Flatten(), # Output layer
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(), metrics = 'accuracy')

In [ ]:
history_1 = model.fit(train_set, epochs=25, validation_data=val_set)

In [ ]:
# Save the model
model.save('d:\\SoftCom_Assignment01\\Model\\cnnModel_dummy.h5')

In [ ]:
tf.random.set_seed(42)

model = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(128,(3,3), activation='relu', input_shape=(256,256,3)), # Hidden Layer 1
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(64,(3,3), activation='relu'), # Hidden Layer 2
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(32,(3,3), activation='relu'), # Hidden Layer 3
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Flatten(), # Output layer
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(), metrics = 'accuracy')

In [ ]:
history_2 = model.fit(train_set, epochs=15, validation_data=val_set)

In [ ]:
# Save the model
model.save('d:\\SoftCom_Assignment01\\Model\\cnnModel_dummy2.h5')

In [ ]:
tf.random.set_seed(42)

model = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16,(3,3), activation='relu', input_shape=(256,256,3)), # Hidden Layer 1
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(32,(3,3), activation='relu'), # Hidden Layer 2
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Flatten(), # Output layer
    layers.Dropout(0.5),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(3, activation='softmax')
])

In [ ]:
pip install optuna

In [ ]:
import optuna
from tensorflow.keras import layers
from tensorflow import keras

def create_model(trial):
    model = keras.Sequential([
        layers.Rescaling(1./255),
        layers.Conv2D(trial.suggest_int('conv1_filters', 32, 128), (3,3), activation='relu', input_shape=(256,256,3)),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(trial.suggest_int('conv2_filters', 32, 128), (3,3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(trial.suggest_int('conv3_filters', 32, 128), (3,3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(trial.suggest_float('dropout1_rate', 0.0, 0.5)),
        layers.Dense(trial.suggest_int('dense1_units', 64, 256), activation='relu'),
        layers.Dropout(trial.suggest_float('dropout2_rate', 0.0, 0.5)),
        layers.Dense(trial.suggest_int('dense2_units', 64, 256), activation='relu'),
        layers.Dense(3, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

def objective(trial):
    # Use your train_set and val_set here
    model = create_model(trial)
    
    history = model.fit(train_set, epochs=15, validation_data=val_set, verbose=0)
    
    return -history.history['val_accuracy'][-1]  # Minimize negative validation accuracy

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

best_params = study.best_params
print("Best Hyperparameters:", best_params)


In [ ]:
from tensorflow.keras import layers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Best hyperparameters from the Optuna trials
best_params = {
    'conv1_filters': 32,
    'conv2_filters': 64,
    'conv3_filters': 128,
    'dropout1_rate': 0.5,
    'dense1_units': 128,
    'dropout2_rate': 0.5,
    'dense2_units': 64,
}

# Define the CNN model using the best hyperparameters
model = tf.keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(best_params['conv1_filters'], (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(best_params['conv2_filters'], (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(best_params['conv3_filters'], (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(best_params['dropout1_rate']),
    layers.Dense(best_params['dense1_units'], activation='relu'),
    layers.Dropout(best_params['dropout2_rate']),
    layers.Dense(best_params['dense2_units'], activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [ ]:
history_optimise_final = model.fit(train_set, epochs=25, validation_data=val_set)

In [ ]:
# Save the model
model.save('d:\\SoftCom_Assignment01\\Model\\cnnModel_optimised_final.h5')

In [ ]:
model_optimised_final = tf.keras.models.load_model('d:\\SoftCom_Assignment01\\Model\\cnnModel_optimised.h5')

In [ ]:
test_score = model_optimised_final.evaluate(test_data, verbose= 1)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
X_test, y_test = None, None
for images, labels in test_data.take(500):
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [ ]:
y_pred_proba = model_optimised_final.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
from tensorflow.keras import layers
# Best hyperparameters obtained from Optuna
best_hyperparameters = {
    'conv1_filters': 121,
    'conv2_filters': 66,
    'conv3_filters': 127,
    'dropout1_rate': 0.3564713596268573,
    'dense1_units': 152,
    'dropout2_rate': 0.15905312684947298,
    'dense2_units': 98
}

# Create the CNN model with the best hyperparameters
model = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(best_hyperparameters['conv1_filters'], (3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(best_hyperparameters['conv2_filters'], (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(best_hyperparameters['conv3_filters'], (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(best_hyperparameters['dropout1_rate']),
    layers.Dense(best_hyperparameters['dense1_units'], activation='relu'),
    layers.Dropout(best_hyperparameters['dropout2_rate']),
    layers.Dense(best_hyperparameters['dense2_units'], activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics='accuracy'
)


In [ ]:
history_optimise = model.fit(train_set, epochs=25, validation_data=val_set)

In [ ]:
# Save the model
model.save('d:\\SoftCom_Assignment01\\Model\\cnnModel_optimised.h5')

In [ ]:
from tensorflow.keras import layers

In [ ]:
for images_batch, labels_batch in train_set:
    print(images_batch.shape)
    print(labels_batch.shape)
    break

In [ ]:
tf.random.set_seed(42)

model = keras.Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16,(3,3), activation='relu', input_shape=(256,256,3)), # Hidden Layer 1
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Conv2D(32,(3,3), activation='relu'), # Hidden Layer 2
    layers.MaxPooling2D(pool_size = (2,2)),
    layers.Flatten(), # Output layer
    layers.Dropout(0.5),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(3, activation='softmax')
])

In [ ]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(), optimizer = keras.optimizers.Adam(), metrics = 'accuracy')

In [ ]:
history = model.fit(train_set, epochs=15, validation_data=val_set)

In [ ]:
# Save the model
model.save('d:\\SoftCom_Assignment01\\Model\\cnnModel.h5')

In [ ]:
model_optimised = tf.keras.models.load_model('d:\\SoftCom_Assignment01\\Model\\cnnModel_optimised.h5')

In [ ]:
model_1 = tf.keras.models.load_model('d:\\SoftCom_Assignment01\\Model\\cnnModel.h5')

In [ ]:
model_optimised.summary()

In [ ]:
def plot_training_curves(history_df):
    plt.figure(figsize = (13, 4), dpi = 120)
    ax = plt.subplot(1, 2, 1)
    plt.plot(range(1, len(history_df) + 1), history_df['loss'], marker = '.', label = 'Training Loss')
    plt.plot(range(1, len(history_df) + 1), history_df['val_loss'], marker = '^', label = 'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Cross Entropy')
    plt.grid()
    plt.legend()
    ax = plt.subplot(1, 2, 2) 
    plt.plot(range(1, len(history_df) + 1), history_df['accuracy'], marker = '.', label = 'Training Accuracy')
    plt.plot(range(1, len(history_df) + 1), history_df['val_accuracy'], marker = '^', label = 'Validation Accurcay')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
plot_training_curves(pd.DataFrame(history_optimise.history))

In [ ]:
X_test, y_test = None, None
for images, labels in test_data.take(200):
    if X_test == None or y_test == None:
        X_test = images
        y_test = labels
    else:
        X_test = tf.concat([X_test, images], axis = 0)
        y_test = tf.concat([y_test, labels], axis = 0)
        
X_test.shape, y_test.shape

In [ ]:
y_pred_proba = model_optimised.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis = 1)

In [ ]:
test_score = model_optimised.evaluate(test_data, verbose= 1)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")

In [ ]:
import seaborn as sns
# Plot confusion matrix
labels = ['covid', 'normal', 'pneumonia']
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# plot random images from a given dataset, and compare predictions with ground truth
def plot_random_predictions(dataset, model_optimised):

    shuffled_data = dataset.shuffle(10)
    class_names = dataset.class_names

    for images, labels in shuffled_data.take(1):
        plt.figure(figsize = (10, 10), dpi = 120)
        y_pred_proba = model_optimised.predict(images)

    for i in range(9):
        index = random.randint(0, len(images))
        ax = plt.subplot(3,3, i + 1)

        img = images[index].numpy().astype("uint8")
        y_true = class_names[labels[index]]
        y_pred = class_names[np.argmax(y_pred_proba[index], axis = 0)]
      
        c = 'g' if y_pred == y_true else 'r'
      
        plt.imshow(img)
        plt.title(f'Predicted : {y_pred}\nTrue label : {y_true}', c = c)
        plt.axis(False)

In [ ]:
plot_random_predictions(test_data, model_1)